### A notebook that sources preprocessed data from the seafloor public bucket and allows local running of GARPOS

In [1]:
#need this first so TILEDB can find it
import os
os.environ['AWS_PROFILE'] = 'prod'

In [ ]:
#imports
from es_sfgtools.data_mgmt.ingestion.archive_pull import load_site_metadata
from es_sfgtools.workflows.workflow_handler import WorkflowHandler
from es_sfgtools.config.env_config import Environment
import pandas as pd
# from earthscope_sdk import AsyncEarthScopeClient
# import tiledb

Golang binaries not built. Navigate to /Users/mikegottlieb/GIT/es_sfgtools/src/golangtools and run "make"
/opt/homebrew/Caskroom/miniconda/base/envs/seafloor_geodesy_mac/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# set network/station/campaign and get metadata
NETWORK = "cascadia-gorda"
STATION = "NCC1"
CAMPAIGN_NAME = "2025_A_1126"

metadata = load_site_metadata(NETWORK, STATION)
metadata

Loading site metadata from https://data.earthscope.org/archive/seafloor/metadata/cascadia-gorda/NCC1.json
Loading vessel metadata from https://data.earthscope.org/archive/seafloor/metadata/vessels/1126.json
Loading vessel metadata from https://data.earthscope.org/archive/seafloor/metadata/vessels/1126.json
Loading vessel metadata from https://data.earthscope.org/archive/seafloor/metadata/vessels/1063.json
Loading vessel metadata from https://data.earthscope.org/archive/seafloor/metadata/vessels/1065.json


Site(names=['NCC1'], networks=['cascadia-gorda'], timeOrigin=datetime.datetime(2021, 10, 1, 0, 0), localGeoidHeight=-27.43, arrayCenter=Location(latitude=41.6569428, longitude=-124.93880652, elevation=-27.43), campaigns=[Campaign(name='2025_A_1126', type='measure', vesselCode='1126', start=datetime.datetime(2025, 9, 7, 20, 27), end=datetime.datetime(2025, 9, 13, 7, 12), vessel=Vessel(name='1126', type='waveglider', model='sv3', serialNumber=None, start=datetime.datetime(2023, 2, 1, 0, 0), end=None, imuSensors=[], atdOffsets=[AtdOffset(serialNumber='641-0654', x=0.0, y=0.00053, z=0.92813)], gnssAntennas=[GnssAntenna(type='novatel', serialNumber='NMLM22110020V', start=datetime.datetime(2023, 2, 28, 0, 0), order='primary', model='GNSS-802L', radomeSerialNumber=None, end=None), GnssAntenna(type='novatel', serialNumber='NMLM22090012Z', start=datetime.datetime(2023, 2, 28, 0, 0), order='secondary', model='GNSS-802L', radomeSerialNumber=None, end=None)], gnssReceivers=[], acousticTransducers=

In [4]:
# print surveys for campaign from metadata
print("Surveys for campaign:", CAMPAIGN_NAME)
for campaign in metadata.campaigns:
    if campaign.name == CAMPAIGN_NAME:
        for survey in campaign.surveys:
            print("  ", survey.id, survey.start, survey.end)

Surveys for campaign: 2025_A_1126
   2025_A_1126_1 2025-09-07 20:27:00 2025-09-08 13:40:00
   2025_A_1126_2 2025-09-08 13:40:00 2025-09-12 19:33:00
   2025_A_1126_3 2025-09-12 19:33:00 2025-09-13 07:12:00


In [5]:
# load the data from S3
S3_SYNC_BUCKET = "seafloor-public-bucket-bucket83908e77-6ial2vrmrawf"
WORKING_ENVIRONMENT_KEY = "GEOLAB"
MAIN_DIRECTORY_GEOLAB_KEY = "/Users/mikegottlieb/data/"
os.environ["S3_SYNC_BUCKET"] = S3_SYNC_BUCKET
os.environ["WORKING_ENVIRONMENT"] = WORKING_ENVIRONMENT_KEY
os.environ["MAIN_DIRECTORY_GEOLAB"] = MAIN_DIRECTORY_GEOLAB_KEY
#os.environ.get(WORKING_ENVIRONMENT_KEY)#, "LOCAL").upper()
env = Environment()
env.load_working_environment()
env.working_environment()
#env.load_aws_credentials()


<WorkingEnvironment.GEOLAB: 'GEOLAB'>

In [6]:
workflow = WorkflowHandler()

workflow.set_network_station_campaign(
    network_id=NETWORK,
    station_id=STATION,
    campaign_id=CAMPAIGN_NAME
)

Changed working station to cascadia-gorda NCC1 2025_A_1126


In [7]:
workflow.midprocess_prep_garpos() 

Network cascadia-gorda already exists.
Station NCC1 already exists in network cascadia-gorda
Campaign 2022_A_1065 already exists in station NCC1
Campaign 2023_A_1063 already exists in station NCC1
Campaign 2024_A_1126 already exists in station NCC1
Campaign 2025_A_1126 already exists in station NCC1


Processing survey 2025_A_1126_1
Processing survey 2025_A_1126_2
Processing survey 2025_A_1126_3


In [ ]:
# print surveys for campaign from metadata
print("Surveys for campaign:", CAMPAIGN_NAME)
for campaign in metadata.campaigns:
    if campaign.name == CAMPAIGN_NAME:
        for survey in campaign.surveys:
            print("  ", survey.id, survey.start, survey.end)
            

In [ ]:
#read shot data dfs from disk
def read_shotdata_file(filepath):
    return pd.read_csv(filepath, sep="\t", header=0)



DirectoryHandler(filepath=PosixPath('/Users/mikegottlieb/data/directoryCatalog.json'), asset_catalog_db_path=PosixPath('/Users/mikegottlieb/data/catalog.sqlite'), networks={'cascadia-gorda': NetworkDir(stations={'NCC1': StationDir(campaigns={'2024_A_1126': CampaignDir(location=PosixPath('/Users/mikegottlieb/data/cascadia-gorda/NCC1/2024_A_1126'), raw=PosixPath('/Users/mikegottlieb/data/cascadia-gorda/NCC1/2024_A_1126/raw'), processed=PosixPath('/Users/mikegottlieb/data/cascadia-gorda/NCC1/2024_A_1126/processed'), intermediate=PosixPath('/Users/mikegottlieb/data/cascadia-gorda/NCC1/2024_A_1126/intermediate'), surveys={'processed': SurveyDir(location=PosixPath('/Users/mikegottlieb/data/cascadia-gorda/NCC1/2024_A_1126/processed'), shotdata=None, kinpositiondata=None, imupositiondata=None, metadata=None, garpos=None, name='processed', campaign=PosixPath('/Users/mikegottlieb/data/cascadia-gorda/NCC1/2024_A_1126'))}, log_directory=PosixPath('/Users/mikegottlieb/data/cascadia-gorda/NCC1/2024_